
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Delta Live Tables Running Modes
We can trigger execution of DLT pipelines in two modes: triggered and continuous. In triggered mode, we can trigger the pipeline manually or schedule the pipeline to run on specific increments. Let's explore these modes more fully.

Run the following setup script to get started.

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:

1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

  - In the drop-down, select **More**.

  - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

1. Find the triangle icon to the right of your compute cluster name and click it.

1. Wait a few minutes for the cluster to start.

1. Once the cluster is running, complete the steps above to select your cluster.

## A. Classroom Setup

Run the following cell to configure your working environment for this course. It will also set your default catalog to **dbacademy** and the schema to your specific schema name shown below using the `USE` statements.
<br></br>


```
USE CATALOG dbacademy;
USE SCHEMA dbacademy.<your unique schema name>;
```

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course.

In [0]:
%run ./Includes/Classroom-Setup-3

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Loading batch 1 of 31...0 seconds


True

**NOTE:** If you have not completed the DLT pipeline from the previous steps (**1a, 1b and 1c**), uncomment and run the following cell to create the pipeline using the solution SQL notebooks to complete this demonstration.

In [0]:
# DA.generate_pipeline(
#     pipeline_name=DA.generate_pipeline_name(), 
#     use_schema = DA.schema_name,
#     notebooks_folder='2A - SQL Pipelines/(Solutions) 2A - SQL Pipelines', 
#     pipeline_notebooks=[
#         '1 - Orders Pipeline',
#         '2 - Customers Pipeline',
#         '3L - Status Pipeline Lab'
#         ],
#     use_configuration = {'source':f'{DA.paths.stream_source}'}
#     )

Run the next cell. We are going to use its job name from the output in the next steps.

In [0]:
DA.print_pipeline_job_info()

labuser10369179_1748065139: Example Pipeline


'labuser10369179_1748065139: Example Pipeline'

## B. Scheduled Execution
DLT pipelines can be triggered on increments from one minute to one month, all at a specific time of day. Additionally, we can trigger alerts for when the pipeline starts, successfully completes, or fails. Follow the steps below to schedule DLT pipeline runs:

1. Find your pipeline using the name from the above cell. Go to your pipeline's configuration page. 

2. In the upper-right corner, drop open the **`Schedule`**. Note the dialog that appears. We are actually going to be creating a job that will have one task, our pipeline

3. In the **Advanced** tab, set the schedule to every month, but note the other options

4. Select the **More options** drop down. Note that we can set notifications for start, success, and failure

5. Click **Create**. A small window appears in the upper-right corner that shows our job was successfully created. The **Schedule** drop-down remains open and shows our current job, and we can add additional schedules, if needed.

6. Select the job link in the pop up. It will take you to the Workflow (job).

7. Click the **Tasks** tab in the upper-left corner

**Our DLT pipeline is the only task for this job**. If we wished, we could configure additional tasks.  
  
The task's configuration fields give us more options for the task.  We can: 
* Trigger a full refresh on the Delta Live Tables pipeline
* Add, or change, notifications
* Configure a retry policy for the task
* Set duration thresholds where we can set times where we want to be warned that a pipeline is taking longer than expected or how long before we should cause the pipeline to timeout.  
  
8. Lastly, there are options on the right side for the whole job. We can see the schedule we created under **Schedules & Triggers**

9. We will not be using the job. Click the "kebab" menu to the left of the **Run now** button, and select **Delete job**

10. Leave the **Workflows** page open.

## C. Continuous Execution
Setting a DLT pipeline to continuous execution will cause the pipeline to run continuously and process data as it arrives in our data sources. To avoid unnecessary processing in continuous execution mode, pipelines automatically monitor dependent Delta tables and perform an update only when the contents of those dependent tables have changed.

Please note: After a pipeline is set to continuous mode and the pipeline is started, a cluster will run continuously until manually stopped. This will add to costs.

To configure a DLT pipeline for continuous execution, complete the following:

1. On the **Workflows** page select **Pipelines** in the navigation bar and select your pipeline.

2. On the pipeline configuration page, click **Settings** at the top right of the page.

3. Under **Pipeline mode**, select **Continuous**

4. Click **Save and start**

The pipeline immediately begins its startup process. This process is very similar to a manually triggered start, except that after the first pipeline run, the pipeline will not shutdown, but will continue to monitor for new data.

Wait for the pipeline to complete before starting the next steps.

## D. Get Current Data from Silver Table


### D1. View the tables in your schema

1. Navigate to the Catalog icon in the vertical bar (directly to the left of the notebook beneath the **File** icon). This will display all of the available catalogs to the left of the notebook. Do not select the **Catalog** text in the main navigation bar on the far left.

2. Expand the **dbacademy** catalog.

3. Expand your unique schema name.

4. Notice that the DLT pipeline created all of the streaming tables and materialized views in the location you specified.

### D2. Query your materialized view
Run the cell below to get the current data for a customer named "Michael Lewis".

Note the current address for *Michael Lewis* is *706 Melissa Canyon*.

In [0]:
%sql
SELECT name, address 
FROM customers_silver 
WHERE name = "Michael Lewis"

name,address
Michael Lewis,706 Melissa Canyon


## E. Let's Add Data
The `DA` object that was created in the Classroom-Setup script we ran at the beginning of this lesson contains a method that will add data to our volume. Let's use this method to see how our continuously running pipeline updates our resulting tables. 

We will be examining our pipeline results more fully in the next lesson. Run the next cell. While the cell is running you can watch your DLT pipeline and view as data is ingested and created.

While the cell is running watch your DLT pipeline. Notice that the numbers are changing on each task.

**Important** - The cell will throw an error if the pipeline has not completed at least one run. If you get an error that the table does not exist, wait for the pipeline to complete one run.

In [0]:
DA.dlt_data_factory.load(continuous=True, delay_seconds=0)

Loading batch 2 of 31...1 seconds
Loading batch 3 of 31...1 seconds
Loading batch 4 of 31...0 seconds
Loading batch 5 of 31...1 seconds
Loading batch 6 of 31...1 seconds
Loading batch 7 of 31...0 seconds
Loading batch 8 of 31...1 seconds
Loading batch 9 of 31...0 seconds
Loading batch 10 of 31...1 seconds
Loading batch 11 of 31...0 seconds
Loading batch 12 of 31...1 seconds
Loading batch 13 of 31...1 seconds
Loading batch 14 of 31...0 seconds
Loading batch 15 of 31...1 seconds
Loading batch 16 of 31...0 seconds
Loading batch 17 of 31...1 seconds
Loading batch 18 of 31...1 seconds
Loading batch 19 of 31...0 seconds
Loading batch 20 of 31...1 seconds
Loading batch 21 of 31...0 seconds
Loading batch 22 of 31...1 seconds
Loading batch 23 of 31...1 seconds
Loading batch 24 of 31...0 seconds
Loading batch 25 of 31...1 seconds
Loading batch 26 of 31...0 seconds
Loading batch 27 of 31...1 seconds
Loading batch 28 of 31...0 seconds
Loading batch 29 of 31...1 seconds
Loading batch 30 of 31...1 s

False

## F. Requery Data
Run the cell below, and note it is the same code that we ran before adding data to our source directory. 

We added 30 files of data to our source directory. This data included updated information for some of our fake customers. Michael Lewis changed his address, so this change should be reflected in our silver-level customers table. 

If you run this query too quickly, the pipeline will not have completed its update, and the address will not have changed. Wait a few seconds, and rerun the query.

The results should show that *Michael Lewis* has an updated address to *451 Hunt Station*.

In [0]:
%sql
SELECT name, address 
FROM customers_silver 
WHERE name = "Michael Lewis"

name,address
Michael Lewis,451 Hunt Station


## G. (REQUIRED!) Stop the Pipeline
If we do not stop the pipeline, it will continue to run indefinitely.

* Stop the pipeline by clicking **`Stop`**


&copy; 2025 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>